## Environment Setup

In [1]:
import os
import subprocess
import sys
import time
import pandas as pd

In [2]:
REGION = 'us-central1'
#ENDPOINT = '<YOUR_ENDPOINT>' e.g. '337dd39580cbcbd2-dot-us-central2.pipelines.googleusercontent.com'
INPUT_FILE = 'gs://input_data_amy_bkt1/Anonymized_Fermentation_Data_final.xlsx'
ARTIFACT_STORE_URI = 'gs://workshop_trial_artifact_store_pp'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

In [3]:
REGION = 'us-central1'
ARTIFACT_STORE = 'gs://workshop_trial_artifact_store_pp'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
DATA_ROOT='{}/data'.format(ARTIFACT_STORE)
JOB_DIR_ROOT='{}/jobs'.format(ARTIFACT_STORE)
TRAINING_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'training', 'X_train.xlsx')
TRAINING_FILE_DIR='{}/{}'.format(DATA_ROOT, 'training')
VALIDATION_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'validation', 'X_validate.xlsx')
VALIDATION_FILE_DIR='{}/{}'.format(DATA_ROOT, 'validation')
TESTING_FILE_DIR='{}/{}'.format(DATA_ROOT, 'testing')

## Step 2a: Write the training APP

In [4]:
TRAINING_APP_FOLDER = 'training_app'
os.makedirs(TRAINING_APP_FOLDER, exist_ok=True)

## Step 2b: Write the training script

In [13]:
%%writefile {TRAINING_APP_FOLDER}/train.py

# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import subprocess
import sys

import fire
import pickle
import numpy as np
import pandas as pd

import json

import pickle
import time


import hypertune

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split  ## from analysis

#simple sklearn impute and scale numeric pipeline
from sklearn.pipeline import Pipeline ## from analysis
from sklearn.impute import SimpleImputer ## from analysis
from sklearn.preprocessing import StandardScaler ## from analysis
import numpy as np ## from analysis

from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import RandomizedSearchCV


import functools


from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

from sklearn import linear_model
from sklearn import model_selection
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from googleapiclient import discovery
from googleapiclient import errors




def train_evaluate(job_dir, input_file, training_dataset, validation_dataset, n_estimators, max_depth, min_samples_leaf, max_features, min_samples_split, class_weight, max_leaf_nodes, random_state, hptune, bootstrap):

   
    obj = input_file
    print("obj", obj)
    data = pd.read_excel(obj,sheet_name='data') 
    print("Opened excel file and assigned to data")
    meta_data = pd.read_excel(obj,sheet_name='meta data') 
    print("Just after file opening")
    
    if not hptune:
        df_train = pd.concat([df_train, df_validation])


    ## Preprocess    
    #Prepare data for analysis
    #Split out numeric from categorical varibles

    ##var_type_filter = [x in ['physiological','biochemical','process'] for x in meta_data['variable type']]
    var_type_filter = [x in ['independent'] for x in meta_data['variable type']]
    var_dtype_filter = (data.dtypes == 'float64') | (data.dtypes == 'int64')

    numeric_vars = (var_type_filter & var_dtype_filter).values
    numeric_x_data = data[data.columns[numeric_vars]]

    #things to try to predict
    y_data = data[data.columns[(meta_data['target'] == 1).values]]

    #meta data about variables
    meta_data = meta_data.query('name in {}'.format(list(data.columns[numeric_vars].values))).set_index('name')
 
    #Variables which will be used to build the model
    ####data.columns[numeric_vars].values
        


    model_target = 'Run_Performance' ## Select target for classification
    
    print("before splitting data")
    #maintain class balance
    X_train, X_test, y_train, y_test = train_test_split(numeric_x_data, y_data, test_size=0.25, stratify = y_data[model_target], random_state=42)

    #split train set to create a pseudo test or validation dataset
    X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=0.33, stratify= y_train[model_target], random_state=42)
    
    
    print('The training, validation and test data contain {}, {} and {} rows respectively'.format(len(X_train),len(X_validate),len(X_test)))

    #save_list_train = ['X_train','y_train','meta_data']
    #save_list_validate = ['X_validate','y_validate','meta_data']
    #save_list_test = ['X_test','y_test','meta_data']

    #for x in save_list_train:
    #    print("training_dataset", training_dataset)
    #    obj = pd.DataFrame(globals()[x])
    #    cmd = "obj.to_csv('{}/{}.csv')".format(training_dataset, x)
    #    eval(cmd)

    #for y in save_list_validate:
    #    print("validation_dataset", validation_dataset)
    #    obj = pd.DataFrame(globals()[y])
    #    cmd = "obj.to_csv('{}/{}.csv')".format(validation_dataset, y)
    #    eval(cmd)  

    #for z in save_list_test:
    #    print("testing_dataset", testing_dataset)
    #    obj = pd.DataFrame(globals()[z])
    #    cmd = "obj.to_csv('{}/{}.csv')".format(testing_dataset, z)
    #    eval(cmd)

    ## Train, optimize and validate predictive model
    ### Train



    classifier = RandomForestClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                min_samples_leaf=min_samples_leaf,
                max_features=max_features,
                min_samples_split=min_samples_split,
                class_weight=class_weight,
                max_leaf_nodes=max_leaf_nodes,
                random_state=random_state,
                bootstrap=bootstrap

     )

    imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    #auto scale
    scaler = StandardScaler()

    estimator = Pipeline([
      ('imputer', imputer),
      ('scaler', scaler),
      ('classifier', classifier),
    ])
    

    #prepare data for modeling
    #use the pipeline created above
    #_X_train = pipe.fit_transform(X_train)
    #_y_train = y_train[model_target]    ## selected target label for prediction
    #_X_test = pipe.fit_transform(X_validate)
    #_y_test = y_validate[model_target]

    #_X_train = pipe.fit_transform(X_train)
    _X_train = X_train    
    _y_train = y_train[model_target]    ## selected target label for prediction
    _X_test = X_validate
    _y_test = y_validate[model_target]
    

    print('Starting training: n_estimators={}, max_depth={}, min_samples_leaf={}, max_features={}, min_samples_split={}, class_weight={}, max_leaf_nodes={}, random_state={}, hptune={}, bootstrap={}'.format(n_estimators, max_depth, min_samples_leaf, max_features, min_samples_split, class_weight, max_leaf_nodes, random_state, hptune, bootstrap))

    #estimator.set_params(classifier__alpha=alpha, classifier__max_iter=max_iter) 
    estimator.set_params(classifier__n_estimators=n_estimators, classifier__max_depth=max_depth, classifier__min_samples_leaf=min_samples_leaf, 
                         classifier__max_features=max_features, classifier__min_samples_split=min_samples_split, classifier__class_weight=class_weight,
                         classifier__max_leaf_nodes=max_leaf_nodes, classifier__random_state=random_state, classifier__bootstrap=bootstrap) 
    #pipeline.fit(X_train, y_train)
    estimator.fit(_X_train, _y_train)

    
    if hptune:
        accuracy = estimator.score(_X_test, _y_test)
        print('Model accuracy: {}'.format(accuracy))
        # Log it with hypertune
        hpt = hypertune.HyperTune()
        hpt.report_hyperparameter_tuning_metric(
          hyperparameter_metric_tag='accuracy',
          metric_value=accuracy
        )

    # Save the model
    if not hptune:
        model_filename = 'model.pkl'
        with open(model_filename, 'wb') as model_file:
            pickle.dump(estimator, model_file)
        gcs_model_path = "{}/{}".format(job_dir, model_filename)
        subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
        print("Saved model in: {}".format(gcs_model_path)) 

    

        
if __name__ == "__main__":
    fire.Fire(train_evaluate)





Overwriting training_app/train.py


## Step 3: Package the script into a docker image.

In [6]:
%%writefile {TRAINING_APP_FOLDER}/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire cloudml-hypertune scikit-learn==0.20.4 pandas==0.24.2
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]


Overwriting training_app/Dockerfile


## Step 4: Build the docker image

In [7]:
IMAGE_NAME='trainer_image'
IMAGE_TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, IMAGE_TAG)

In [8]:
!gcloud builds submit --tag $IMAGE_URI $TRAINING_APP_FOLDER

Creating temporary tarball archive of 21 file(s) totalling 105.4 KiB before compression.
Uploading tarball of [training_app] to [gs://etl-project-datahub_cloudbuild/source/1599254137.08997-248e7a5e3b0b46b685f30d3a09b6c756.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/etl-project-datahub/builds/5e998e48-afcc-4ffb-ad9f-9f02eebfabc0].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/5e998e48-afcc-4ffb-ad9f-9f02eebfabc0?project=448067079266].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "5e998e48-afcc-4ffb-ad9f-9f02eebfabc0"

FETCHSOURCE
Fetching storage object: gs://etl-project-datahub_cloudbuild/source/1599254137.08997-248e7a5e3b0b46b685f30d3a09b6c756.tgz#1599254137590293
Copying gs://etl-project-datahub_cloudbuild/source/1599254137.08997-248e7a5e3b0b46b685f30d3a09b6c756.tgz#1599254137590293...
/ [1 files][ 26.2 KiB/ 26.2 KiB]                                                
Operation completed over

# Run hyperparameter tuning jobs

In [9]:
%%writefile {TRAINING_APP_FOLDER}/hptuning_config.yaml

# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


trainingInput:
  hyperparameters:
    goal: MAXIMIZE
    maxTrials: 4
    maxParallelTrials: 4
    hyperparameterMetricTag: accuracy
    enableTrialEarlyStopping: TRUE
    algorithm: RANDOM_SEARCH
    params:
    - parameterName: n_estimators
      type: INTEGER
      minValue: 10
      maxValue: 200
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: max_depth
      type: INTEGER
      minValue: 3
      maxValue: 100
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: min_samples_leaf
      type: INTEGER
      minValue: 10
      maxValue: 500
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: max_features
      type: DOUBLE
      minValue: 0.5
      maxValue: 1.0
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: min_samples_split
      type: DISCRETE
      discreteValues: [
          2,
          5,
          10
      ]
    - parameterName: class_weight
      type: CATEGORICAL
      categoricalValues: [
          "balanced",
          "balanced_subsample"
      ]
    - parameterName: max_leaf_nodes
      type: INTEGER
      minValue: 10
      maxValue: 500
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: random_state
      type: INTEGER
      minValue: 35
      maxValue: 75
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: bootstrap
      type: CATEGORICAL
      categoricalValues: [
          "TRUE",
          "FALSE"
      ]

Overwriting training_app/hptuning_config.yaml


### Step 2: Start the hyperparameter tuning job

job_dir -> GCS Path for storing the job packages & model.

training_dataset_path -> GCS path holding training dataset.

validation_dataset_path -> GCS path holding validation dataset.

alpha -> hyperparameter

max_iter -> hyperparameter

hptune -> variable to decide if hyperparameter tuning is to be done or not.


In [10]:
REGION = 'us-central1'
ARTIFACT_STORE = 'gs://workshop_trial_artifact_store_pp'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
DATA_ROOT='{}/data'.format(ARTIFACT_STORE)
JOB_DIR_ROOT='{}/jobs'.format(ARTIFACT_STORE)
TRAINING_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'training', 'X_train.xlsx')
TRAINING_FILE_DIR='{}/{}'.format(DATA_ROOT, 'training')
VALIDATION_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'validation', 'X_validate.xlsx')
VALIDATION_FILE_DIR='{}/{}'.format(DATA_ROOT, 'validation')
TESTING_FILE_DIR='{}/{}'.format(DATA_ROOT, 'testing')

In [11]:
REGION = 'us-central1'
ARTIFACT_STORE = 'gs://workshop_trial_artifact_store_pp'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
DATA_ROOT='{}/data'.format(ARTIFACT_STORE)
JOB_DIR_ROOT='{}/jobs'.format(ARTIFACT_STORE)
#TRAINING_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'training', 'dataset.xlsx')
#VALIDATION_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'validation', 'dataset.xlsx')

In [12]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
JOB_DIR = "{}/{}".format(JOB_DIR_ROOT, JOB_NAME)
SCALE_TIER = "BASIC"
  

!gcloud ai-platform jobs submit training $JOB_NAME \
--region=$REGION \
--job-dir=$JOB_DIR \
--master-image-uri=$IMAGE_URI \
--scale-tier=$SCALE_TIER \
--config $TRAINING_APP_FOLDER/hptuning_config.yaml \
-- \
--training_dataset=$TRAINING_FILE_DIR \
--validation_dataset=$VALIDATION_FILE_DIR \
--input_file=$INPUT_FILE \
--hptune

Job [JOB_20200904_212118] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20200904_212118

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20200904_212118
jobId: JOB_20200904_212118
state: QUEUED


In [13]:
!gcloud ai-platform jobs describe $JOB_NAME

createTime: '2020-09-04T20:35:21Z'
etag: _zwYn855T4M=
jobId: JOB_20200904_203519
startTime: '2020-09-04T20:35:23Z'
state: RUNNING
trainingInput:
  args:
  - --training_dataset=gs://workshop_trial_artifact_store_pp/data/training
  - --validation_dataset=gs://workshop_trial_artifact_store_pp/data/validation
  - --input_file=gs://input_data_amy_bkt1/Anonymized_Fermentation_Data_final.xlsx
  - --hptune
  hyperparameters:
    algorithm: RANDOM_SEARCH
    enableTrialEarlyStopping: true
    goal: MAXIMIZE
    hyperparameterMetricTag: accuracy
    maxParallelTrials: 4
    maxTrials: 4
    params:
    - maxValue: 200.0
      minValue: 10.0
      parameterName: n_estimators
      scaleType: UNIT_LINEAR_SCALE
      type: INTEGER
    - maxValue: 100.0
      minValue: 3.0
      parameterName: max_depth
      scaleType: UNIT_LINEAR_SCALE
      type: INTEGER
    - maxValue: 500.0
      minValue: 10.0
      parameterName: min_samples_leaf
      scaleType: UNIT_LINEAR_SCALE
      type: INTEGER
    - ma

In [14]:
# for streaming the logs
#!gcloud ai-platform jobs stream-logs $JOB_NAME

^C


Command killed by keyboard interrupt



In [15]:
from googleapiclient import discovery
from googleapiclient import errors
ml = discovery.build('ml', 'v1')

job_id = 'projects/{}/jobs/{}'.format(PROJECT_ID, JOB_NAME)
request = ml.projects().jobs().get(name=job_id)

try:
    response = request.execute()
except errors.HttpError as err:
    print(err)
except:
    print("Unexpected error")
    
response


{'jobId': 'JOB_20200904_203519',
 'trainingInput': {'args': ['--training_dataset=gs://workshop_trial_artifact_store_pp/data/training',
   '--validation_dataset=gs://workshop_trial_artifact_store_pp/data/validation',
   '--input_file=gs://input_data_amy_bkt1/Anonymized_Fermentation_Data_final.xlsx',
   '--hptune'],
  'hyperparameters': {'goal': 'MAXIMIZE',
   'params': [{'parameterName': 'n_estimators',
     'minValue': 10,
     'maxValue': 200,
     'type': 'INTEGER',
     'scaleType': 'UNIT_LINEAR_SCALE'},
    {'parameterName': 'max_depth',
     'minValue': 3,
     'maxValue': 100,
     'type': 'INTEGER',
     'scaleType': 'UNIT_LINEAR_SCALE'},
    {'parameterName': 'min_samples_leaf',
     'minValue': 10,
     'maxValue': 500,
     'type': 'INTEGER',
     'scaleType': 'UNIT_LINEAR_SCALE'},
    {'parameterName': 'max_features',
     'minValue': 0.5,
     'maxValue': 1,
     'type': 'DOUBLE',
     'scaleType': 'UNIT_LINEAR_SCALE'},
    {'parameterName': 'min_samples_split',
     'type'